https://github.com/aws/amazon-sagemaker-examples/blob/4534bff4b5b5062af5789d98c4ddca01b0cb5d1f/end_to_end/fraud_detection/0-AutoClaimFraudDetection.ipynb

In [5]:
import sagemaker 
import boto3

In [7]:
sess = sagemaker.Session()
REGION = sess.boto_region_name

s3_client = boto3.client("s3")
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
print(REGION)

us-east-1


In [13]:
"""
Note: if you are not running this notebook from SageMaker Studio or SageMaker Classic Notebooks you will need to instanatiate 
the sagemaker_execution_role_name with an AWS role that has SageMakerFullAccess and SageMakerFeatureStoreFullAccess
"""
try:
    ROLE = sagemaker.get_execution_role()
    print(ROLE)
except:
    print("Need to set role manually")
    exec_role_name = "AmazonSageMaker-ExecutionRole-20250401T145997"
    sagemaker_role = iam_client.get_role(RoleName=exec_role_name)["Role"]["Arn"]
    print(f"\n instantiating sagemaker_role with supplied role name : {sagemaker_role}")

account_id = boto3.client("sts").get_caller_identity()["Account"]
print(account_id)

arn:aws:iam::205930620783:role/service-role/AmazonSageMaker-ExecutionRole-20250401T145997
205930620783


In [22]:
# Create a directory in the SageMaker default bucket 
BUCKET = sess.default_bucket()
print(BUCKET)
PREFIX="autoclaims-fraud-detection"
# Magic command to store the variable in store.pkl
%store BUCKET
%store PREFIX 
print("creating bucket:")

sagemaker-us-east-1-205930620783
Stored 'BUCKET' (str)
Stored 'PREFIX' (str)
creating bucket:


In [24]:
# Upload Files to S3 Buckets
s3_client.upload_file(Filename="Data/customers.csv", Bucket=BUCKET, Key=f"{PREFIX}/data/raw/customers.csv")
s3_client.upload_file(Filename="Data/claims.csv", Bucket=BUCKET, Key=f"{PREFIX}/data/raw/claims.csv")

In [25]:
claims_flow_template_file = "claims_flow_template"

with open(claims_flow_template_file, "r") as f:
    variables = {"bucket": bucket, "prefix": prefix}
    template = string.Template(f.read())
    claims_flow = template.substitute(variables)
    claims_flow = json.loads(claims_flow)

with open("claims.flow", "w") as f:
    json.dump(claims_flow, f)

customers_flow_template_file = "customers_flow_template"

with open(customers_flow_template_file, "r") as f:
    variables = {"bucket": bucket, "prefix": prefix}
    template = string.Template(f.read())
    customers_flow = template.substitute(variables)
    customers_flow = json.loads(customers_flow)

with open("customers.flow", "w") as f:
    json.dump(customers_flow, f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 claims_flow_template_file = "claims_flow_template"                                          │
│    2                                                                                             │
│ ❱  3 with open(claims_flow_template_file, "r") as f:                                             │
│    4 │   variables = {"bucket": bucket, "prefix": prefix}                                        │
│    5 │   template = string.Template(f.read())                                                    │
│    6 │   claims_flow = template.substitute(variables)                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:324 in _modified_open   │
│                                                                                                  │
│    321 │   │   │   "you can use builtins' open."                                                 │
│    322 │   │   )                                                                                 │
│    323 │                                                                                         │
│ ❱  324 │   return io_open(file, *args, **kwargs)                                                 │
│    325                                                                                           │
│    326 class InteractiveShell(SingletonConfigurable):                                            │
│    327 │   """An enhanced, interactive shell for Python."""                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: 'claims_flow_template'